<a href="https://colab.research.google.com/github/gulabpatel/ML_Algos/blob/main/MultiLabelClassification/01_Multi_label_classification_texts.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/content/drive/MyDrive/datasets/multilabel/dataset-tags.csv")
df.head()

,title,tags,mysql,python,php
0,Flask-SQLAlchemy - When are the tables/databas...,"['python', 'mysql']",1,1.0,0.0
1,Combining two PHP variables for MySQL query,"['php', 'mysql']",1,0.0,1.0
2,'Counting' the number of records that match a ...,"['php', 'mysql']",1,0.0,1.0
3,Insert new row in a table and auto id number. ...,"['php', 'mysql']",1,0.0,1.0
4,Create Multiple MySQL tables using PHP,"['php', 'mysql']",1,0.0,1.0


In [3]:
df.dtypes

title      object
tags       object
mysql       int64
python    float64
php       float64
dtype: object

In [4]:
df['mysql'] = df['mysql'].astype(float)
df.dtypes

title      object
tags       object
mysql     float64
python    float64
php       float64
dtype: object

In [5]:
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
!pip install scikit-multilearn neattext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset

import neattext as nt
import neattext.functions as nfx

In [8]:
# df['title'].apply(lambda x:nt.TextFrame(x).noise_scan())
# df['title'].apply(lambda x:nt.TextExtractor(x).extract_stopwords())
# df['title'].apply(nfx.remove_stopwords)

# corpus = df['title'].apply(nfx.remove_stopwords)
# tfidf = TfidfVectorizer()
# Xfeatures = tfidf.fit_transform(corpus).toarray()
# Xfeatures

In [9]:
def features(df):
  corpus = df.apply(nfx.remove_stopwords)
  tfidf = TfidfVectorizer()
  Xfeatures = tfidf.fit_transform(corpus).toarray()
  return Xfeatures
Xfeatures = features(df['title'])

In [10]:
y = df[['mysql', 'python', 'php']]
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,y,test_size=0.3,random_state=42)
binary_rel_clf = BinaryRelevance(MultinomialNB())
binary_rel_clf.fit(X_train,y_train)

# BinaryRelevance(classifier=MultinomialNB(alpha=1.0, class_prior=None,
#                                          fit_prior=True),
#                 require_dense=[True, True])

BinaryRelevance(classifier=MultinomialNB(), require_dense=[True, True])

In [11]:
br_prediction = binary_rel_clf.predict(X_test)
accuracy_score(y_test,br_prediction), hamming_loss(y_test,br_prediction)

(0.9090909090909091, 0.06060606060606061)

In [12]:
def build_model(model,mlb_estimator,xtrain,ytrain,xtest,ytest):
    clf = mlb_estimator(model)
    clf.fit(xtrain,ytrain)
    clf_predictions = clf.predict(xtest)
    acc = accuracy_score(ytest,clf_predictions)
    ham = hamming_loss(ytest,clf_predictions)
    result = {"accuracy:":acc,"hamming_score":ham}
    return result

## Classifier Binary Relevance technique

In [13]:
clf_BinaryRel_model = build_model(MultinomialNB(),BinaryRelevance,X_train,y_train,X_test,y_test)
clf_BinaryRel_model

{'accuracy:': 0.9090909090909091, 'hamming_score': 0.06060606060606061}

## Classifier chains technique

In [14]:
clf_chain_model = build_model(MultinomialNB(),ClassifierChain,X_train,y_train,X_test,y_test)
clf_chain_model

{'accuracy:': 0.8409090909090909, 'hamming_score': 0.10606060606060606}

## Labelpowerset technique

In [15]:
clf_labelP_model = build_model(MultinomialNB(),LabelPowerset,X_train,y_train,X_test,y_test)
clf_labelP_model

{'accuracy:': 0.9090909090909091, 'hamming_score': 0.06060606060606061}